In [20]:
import spacy
import requests
import json
en_nlp = spacy.load('en_core_web_md')
import json
import pickle

sentence = "Jack gave the book to Mary. It was easy. He was scared she would be angry. Ray also thought the same. He had a bad temper. "    
 

def get_gender(payload, prop_noun_entities):
    
    gender_req = requests.get("https://api.genderize.io/", params=payload) 
    json_gen = gender_req.json()
    #print(json_gen)
     
    for ji in range(0, len(json_gen)): 
        name = json_gen[ji][u'name']
        gender = json_gen[ji][u'gender'] 
        prop_noun_entities[name] = gender
     
    return prop_noun_entities 

def get_named_entities(en_doc): 
    prop_noun_entities = {} 
    prop_noun_entities_pos = {} 
    payload = {} 
    i = 0 
    for ent in en_doc.ents: 
        #print(ent)
        if ent.text.split(" ")[0] not in prop_noun_entities_pos.keys():
            prop_noun_entities_pos[ent.text.split(" ")[0]] = ent.start 
        if i < 10: 
            payload["name["+str(i)+"]"] = ent.text.split(" ")[0] 
            
        if i == 9: 
            #print (payload)
            prop_noun_entities = get_gender(payload, prop_noun_entities) 
            i = 0 
        i += 1 
        
    if i < 10:     
        prop_noun_entities = get_gender(payload, prop_noun_entities) 
    
    for token in en_doc:
        #print(token.pos_)
        if token.pos_ =="NOUN":
            prop_noun_entities_pos[token.text] = token.i
    
    for token in en_doc:
        if token.pos_ =="NOUN":
            prop_noun_entities[token.text] = "none"
    
    #print(prop_noun_entities)
    return prop_noun_entities, prop_noun_entities_pos 
  
def propogate_anaphora(en_doc, anaphora_mappings, prop_noun_entities_pos): 
    first_person_pron = ["i", "me", "my", "mine"] 

 
    anaphora_pronouns = list(anaphora_mappings.values()) 
    anaphora_pnouns = list(anaphora_mappings.keys()) 
    #print(anaphora_pronouns)
    #print(anaphora_pnouns)
    
    doc_list = [str(tok) for tok in en_doc] 
    
    prev_noun = ""
    for sent in en_doc.sents: 
         
        for token in sent: 
            #print(token.text) 
            if token.tag_ == "NNP" or token.tag_ == "NNPS": 
                if token.text in anaphora_pnouns:
                    prev_noun = token.text 
            if token.tag_ == "PRP" or token.tag_ == "PRP$": 
                for pos in range(0, len(anaphora_pronouns)): 
                    if str(token.text).lower() in anaphora_pronouns[pos]: 
                        resolve_noun = anaphora_pnouns[pos] 
                        #print(resolve_noun, prev_noun, token.text) 
                        res_pos = prop_noun_entities_pos[resolve_noun]
                        #print (token.i, res_pos)
                        if res_pos < token.i and str(token.text).lower() in first_person_pron: 
                            doc_list[token.i] = prev_noun 
                            #print ('1')
                            break 
                        elif res_pos < token.i and resolve_noun != prev_noun: 
                            if str(token.text).lower() not in anaphora_mappings[prev_noun]:
                                doc_list[token.i] = resolve_noun 
                                prev_noun = resolve_noun
                                #print (anaphora_mappings[prev_noun])
                                break
                            else:
                                doc_list[token.i] = prev_noun
                                
                            break
                        elif res_pos < token.i and resolve_noun == prev_noun:
                            doc_list[token.i] = resolve_noun
                            #print('3')
                            break 
                                   

    return doc_list 

def map_entity_pronoun(prop_noun_entities, entity, anaphora_mappings): 
    pronouns = ["i", "you", "he", "she", "it", "we", "they", "me", "him", "her", "us", "them"] 
    
 
    first_person_pron = ["i", "me", "my", "mine"] 
    second_person_pron = ["you", "your"] 
    third_person_pron = [["he", "him", "his"], ["she", "her", "hers"], ["it"]]  # M - F - N 
 
 
    first_person_plu_pron = ["we", "us", "our", "ours"] 
    second_person_plu_pron = ["you", "yours"] 
    third_person_plu_pron = ["they", "them", "their", "theirs"] 

 
    gender = prop_noun_entities[entity] 

 
    if gender == 'male': 
        anaphora_mappings[entity] = first_person_pron + second_person_pron + third_person_pron[0] 
    elif gender == 'female': 
        anaphora_mappings[entity] = first_person_pron + second_person_pron + third_person_pron[1] 
    else: 
        anaphora_mappings[entity] = third_person_pron[2] 
    return anaphora_mappings 

if __name__ == "__main__": 
    corpusFile = open("corpus.txt","r") 
    data = corpusFile.read() 
    corpusFile.close() 
    #data = data.split("." or "!" or "?")
    
    en_doc = en_nlp(u'' + data) 
    prop_noun = "" 
    anaphora_mappings = {} 
 
    prop_noun_entities, prop_noun_entities_pos = get_named_entities(en_doc)
    
    for entity in prop_noun_entities.keys(): 
        anaphora_mappings = map_entity_pronoun(prop_noun_entities, entity, anaphora_mappings) 
    print(anaphora_mappings)
    resolved_sent = propogate_anaphora(en_doc, anaphora_mappings, prop_noun_entities_pos)
    print(' '.join(resolved_sent))
    '''
    f= open("resolved.txt","w")
    f.write(' '.join(resolved_sent))
    f.close()
    '''



    



KeyError: 0